In [ ]:
#Markdown的部分都已经转移到dorado上完成，此处只负责训练部分，如果要更改数据选取，取消Markdown后更改

#!pip3 install -U http://tosv.byted.org/obj/turing-us/byted_turing-0.3.0-py3-none-any.whl

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Embedding, Input
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import Callback
import random as rd
from math import floor
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix

#读取要分析的事件和字段
events=lark_sdk.read_excel(url='https://bytedance.feishu.cn/sheets/shtcnZ4iIvOh7UmEO1OYuY39gDe?sheet=EqrTnO', sheet='Sheet12',header=True).values.tolist()

#解析字段并分类
usu=set(events[0][1].split('\n'))
unu=set(events[1][1].split('\n'))
eve=[[i[0],set(i[1].split('\n')).union(usu).difference(unu)] for i in events][2:]

al=set()
for i in eve:
    al=al.union(i[1])
len(list(al)),len([i[0] for i in events])
useful="'"+"','".join([j[0] for j in eve])+"'"
params="'"+"','".join(al)+"'"
usual="'"+"','".join(list(usu))+"'"

#分析数据库中的字段情况
ana="""
SELECT event, a.k as k,count(distinct cast(a.v as bigint))as v,sum(cast(a.v as bigint))/count(a.v) as avg,min(cast(a.v as bigint)) as mn,max(cast(a.v as bigint))as ma
FROM mobile_game.dwd_event_log_stage_hi
LATERAL VIEW explode(params)a as k,v
WHERE p_date='2020-12-21'
AND app_id=2447
AND app_id_str='2447'
AND server_id=30016
AND event IN({0})
AND a.k IN({1})
GROUP BY event, a.k
UNION
SELECT 'usual', a.k as k,count(distinct cast(a.v as bigint))as v,sum(cast(a.v as bigint))/count(a.v) as avg,min(cast(a.v as bigint)) as mn,max(cast(a.v as bigint))as ma
FROM mobile_game.dwd_event_log_stage_hi
LATERAL VIEW explode(params)a as k,v
WHERE p_date='2020-12-21'
AND app_id=2447
AND app_id_str='2447'
AND server_id=30016
AND event IN({0})
AND a.k IN({2})
GROUP BY a.k
""".format(useful,params,usual)

pms_ana=spark.sql(ana).toPandas()

pms_ana=pms_ana.query('avg>-999 and v>1')
pms_ana.sort_values('event')

spark.createDataFrame(pms_ana).write.format('hive').mode('overwrite').saveAsTable('oasis_p2_va.dim_p2_key_value')

ana="""
SELECT * FROM oasis_p2_va.dim_p2_key_value
"""

pms_ana=spark.sql(ana).toPandas()

pms_ana.query("event=='usual'")

#总结字段情况，在darado中即生成个人矩阵（非稀疏），notebook中不对个人矩阵再进行操作，否则速度极慢。但这种办法会导致大量数据冗余，导致数据量上限大幅降低
pms1=pms_ana.sort_values('event').values.tolist()
pms1=[i for i in pms1 if i[2]>1]
public=[(n[1],(n[4],n[5])) for n in pms_ana.query("event=='usual'").values.tolist()]
private=[(i[1],(i[4],i[5]),i[0]) for i in pms1 if not i[1] in usu]
public_str=','.join(["(cast(params['{0}'] as bigint)-({1}))/({2}-({1}))".format(i[0],i[1][0],i[1][1]) for i in public])
private_str=','.join(["if(event='{3}',(cast(params['{0}'] as bigint)-({1}))/({2}-({1})),0)".format(i[0],i[1][0],i[1][1],i[2]) for i in private])
params="array({0},{1})".format(public_str,private_str)
params

#总结字段情况，准备生成sql语句，这是稀疏矩阵路线，后弃用，因为处理稀疏矩阵仍然耗时，keras也需要array输入而非稀疏阵
pms1=pms_ana.sort_values('event').values.tolist()
pms1=[i for i in pms1 if i[2]>1]
iiter=0
def itering():
    global iiter
    iiter+=1
    return iiter
public=[(n[1],(n[4],n[5])) for n in pms_ana.query("event=='usual'").values.tolist()]
public_str=','.join(["{3},cast((cast(params['{0}'] as DECIMAL(22,1))-({1}))/{4} as float)".format(i[0],i[1][0],i[1][1],itering(),i[1][1]-i[1][0]) for i in public])
private=[(itering(),i[1],i[4],i[5],i[0]) for i in pms1 if not i[1] in usu]
prev=''
lst=[]
lst1=[]
for i in private:
    if prev=='':prev=i[4]
    lst.append("{0},cast((cast(params['{1}'] as DECIMAL(22,1))-({2}))/{3} as float)".format(i[0],i[1],i[2],i[3]-i[2]))
    if i[4]!=prev:
        lst1.append("WHEN event='{0}' THEN map(".format(prev)+public_str+','+','.join(lst)+")")
        prev=i[4]
        lst=[]
pstr=' '.join(lst1)
PARAMS=pstr

#dim=iiter
dim=180
dim

#读取训练数据
sql="""
SELECT a.role_id, max(if(d.role_id is null,0,1))ret, collect_list(params) pms
FROM(
    SELECT role_id, p_date
    FROM mobile_game.dwd_event_log_stage_hi
    WHERE p_date BETWEEN '{2}' AND '{3}'
    AND app_id=2447
    AND app_id_str='2447'
    AND event='create_role_success'
    AND server_id>=30016
)a
INNER JOIN(
    SELECT role_id,event,time, p_date,
    {0} params,
    row_number() over(
        partition by role_id,p_date
        order by time
        desc
    )rn
    FROM mobile_game.dwd_event_log_stage_hi
    WHERE p_date BETWEEN '{2}' AND '{3}'
    AND app_id=2447
    AND app_id_str='2447'
    AND server_id>=30016
    AND event IN({1})
)b
ON a.role_id=b.role_id AND a.p_date=b.p_date
LEFT JOIN(
    SELECT role_id
    FROM mobile_game.dwd_event_log_stage_hi
    WHERE p_date BETWEEN '{4}' AND '{5}'
    AND app_id=2447
    AND app_id_str='2447'
    AND event='retention'
    AND server_id>=30016
    AND params['day']='1'
    GROUP BY role_id
)d
ON a.role_id=d.role_id
WHERE rn<1200
GROUP BY a.role_id
""".format(params,useful,'2020-12-21','2021-01-26','2020-12-22','2021-01-27')
"""这个代码在dorado分日期运行更快（不要在这里运行这个代码），这里只是sql代码生成，把p_date的范围改为${DATE}"""

In [ ]:
%%hql
SELECT *
FROM oasis_p2_va.dwd_p2_samples_di
WHERE p_date='${DATE}'

In [ ]:
date_list=_

#样本量
len(date_list)

plt.scatter([len(i[1].pms) for i in dfs.iterrows()],[i[1].ret for i in dfs.iterrows()],s=0.1)

tot=dfs.shape[0]
cnt=0
samples=[]
#list-dictionary to sparse matrix
def ld2sm(ld):
    ld=ld[1].pms
    global cnt,tot
    mat=lil_matrix((1000,180))
    for i in range(1000):
        j=999-i
        if i==len(ld):break
        for k in ld[i]:
            mat[j,k]=ld[i][k]
            if not mat[j,k]>-999:mat[j,k]=0
    if floor(100*len(samples)/tot)!=floor(100*(len(samples)+1)/tot):print(floor(100*(len(samples)+1)/tot),flush=True,end=' ')
    samples.append(mat)

from multiprocessing.dummy import Pool
pool=Pool()
pool.map(print,[1,2,3,4,list(range(10000)),6,7,8])
pool.close()
pool.join()

sample0=[[(i[1].pms,i[1].ret) for i in j[2].iterrows()] for j in date_list if j[2] is not None]
samples=[]
y=[]
zero=[0 for i in range(180)]
for i in sample0:
    for j in i:
        j[0].reverse()
        if len(j[0])<1199:
            a=[zero for i in range(1199-len(j[0]))]
            a.extend(j[0])
            samples.append(a)
        else:samples.append(j[0])
        y.append(j[1])

len(samples)

In [ ]:
dim=180
sample0=[(i[1].pms,i[1].ret) for i in date_list.iterrows()]
samples=[]
y=[]
zero=[0 for i in range(180)]
for j in sample0:
    j[0].reverse()
    if len(j[0])<1199:
        a=[zero for i in range(1199-len(j[0]))]
        a.extend(j[0])
        samples.append(a)
    else:samples.append(j[0])
    y.append(j[1])
        
#模型与损失
def create_model(sequence_length, converter=10, units=64, cell=LSTM, n_layers=2, dropout=0.3,
                loss="binary_crossentropy", optimizer="rmsprop"):
    model = Sequential()
    #model.add(Input(shape=(sequence_length,dim),sparse=True))
    #model.add(Embedding(dim+3, converter))
    for i in range(n_layers):
        if i == 0:
            # first layer
            model.add(cell(units, return_sequences=True,input_shape=(sequence_length,dim)))
        elif i == n_layers - 1:
            # last layer
            model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss=loss, metrics=["binary_accuracy"], optimizer=optimizer)
    model.summary()
    return model

class Metrics(Callback):

    def __init__(self, val_data, batch_size = 20):
        super().__init__()
        self.validation_data = val_data
        self.batch_size = batch_size

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(
            self.validation_data[0]))).round().astype(int)
        val_targ = self.validation_data[1].astype(int)
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        _val_confusion = confusion_matrix(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(_val_confusion)

        return
    
#参数
N_STEPS = None
N_LAYERS = 3
# LSTM cell
CELL = LSTM
# 256 LSTM neurons
UNITS = 128
# 40% dropout
DROPOUT = 0.4


LOSS = "binary_crossentropy"
OPTIMIZER = "adam"
BATCH_SIZE = 128
EPOCHS = 20

ret=[]
los=[]
for i in range(len(y)):
    if y[i]==1:ret.append(i)
    else:los.append(i)
los=los[:len(ret)]
ret.extend(los)
rd.shuffle(ret)

model = create_model(N_STEPS, 40, loss=LOSS, units=UNITS, cell=CELL, n_layers=N_LAYERS,
                    dropout=DROPOUT, optimizer=keras.optimizers.Adam(clipvalue=0.5,lr=0.001))
cut=round(len(ret)*0.1)
v_dat=ret[:cut]
t_dat=ret[cut:]

X_train=np.asarray([samples[i] for i in t_dat])
Y_train=np.asarray([y[i] for i in t_dat])
X_val=np.asarray([samples[i] for i in v_dat])
Y_val=np.asarray([y[i] for i in v_dat])

In [ ]:
print(X_train.shape)

metrics=Metrics((X_val,Y_val),batch_size=BATCH_SIZE)
history=model.fit(X_train,Y_train,initial_epoch=0,epochs=50,batch_size=BATCH_SIZE,shuffle=True,
          validation_data=(X_val,Y_val),callbacks=[metrics])

print(metrics.val_precisions)
print(metrics.val_recalls)
import time
model.save('loss{0}.h5'.format(time.time()))